In [3]:
import geohash
import pandas as pd #数据分析
import numpy as np #科学计算
from pandas import Series,DataFrame
from datetime import datetime
import matplotlib.pyplot as plt
import csv
"""=====================================================================================================================
1 读取数据
"""
path ="E:/data2/"

data_operation = pd.read_csv(path + "operation_train_new.csv")

data_tag=pd.read_csv(path + "tag_train_new.csv")
test_operation = pd.read_csv(path + "operation_round1_new.csv")
#train_operation = pd.merge(data_operation, data_tag, how='left', left_on='UID',right_on='UID')
"""=====================================================================================================================
2 将每一行空的数量作为特征，加到每一行
"""

"""【训练集】"""
temp1=train_operation.isnull()
num=(temp1 == True).astype(bool).sum(axis=1)
is_null=DataFrame(list(zip(num)))
is_null=is_null.rename(columns={0:"is_null_num"})
train_operation = pd.merge(train_operation,is_null,left_index = True, right_index = True, how = 'outer')
"""【测试集】"""
temp1=test_operation.isnull()
num=(temp1 == True).astype(bool).sum(axis=1)
is_null=DataFrame(list(zip(num)))
is_null=is_null.rename(columns={0:"is_null_num"})
test_operation = pd.merge(test_operation,is_null,left_index = True, right_index = True, how = 'outer')
"""=====================================================================================================================
3 time时间归一化 小时
"""
"""【训练集】"""
train_operation['time'] = pd.to_datetime(train_operation['time'])
time_now = train_operation['time'].apply(lambda x:int((x-datetime(2018,10,30,0,0,0)).seconds/3600))
train_operation['time']= time_now

"""【测试集】"""
test_operation['time'] = pd.to_datetime(test_operation['time'])
time_now = test_operation['time'].apply(lambda x:int((x-datetime(2018,10,30,0,0,0)).seconds/3600))
test_operation['time']= time_now
"""=====================================================================================================================
4 mode的处理
"""
n1=set(train_operation['mode'])
n2=set(test_operation['mode'])
n=n1.union(n2)
dic={}
for i,j in enumerate(n):
    dic[j]=i
"""【训练集】"""
train_operation['mode'] = train_operation['mode'].map(dic)

"""【测试集】"""
test_operation['mode'] = test_operation['mode'].map(dic)
"""=====================================================================================================================
5 success的处理,缺失值置为3
"""
"""【训练集】"""
train_operation['success'] = train_operation['success'].fillna(3)
train_operation['success'] = train_operation.success.apply(lambda x:float(x))

"""【测试集】"""
test_operation['success'] = test_operation['success'].fillna(3)
test_operation['success'] = test_operation.success.apply(lambda x:float(x))
"""=====================================================================================================================
6 version的处理,缺失值为0
"""
train_operation['version'] = train_operation['version'].fillna(0)
test_operation['version'] = test_operation['version'].fillna(0)
n1=set(train_operation['version'])
n2=set(test_operation['version'])
n=n1.union(n2)
dic={}
for i,j in enumerate(n):
    dic[j]=i
"""【训练集】"""
train_operation['version'] = train_operation['version'].map(dic)

"""【测试集】"""
test_operation['version'] = test_operation['version'].map(dic)
"""=====================================================================================================================
7 device1和device2的处理,暂时先删掉，后续可能对手机型号进行分类
"""
"""【训练集】"""
train_operation.drop(['device1','device2'],axis=1,inplace=True)

"""【测试集】"""
test_operation.drop(['device1','device2'],axis=1,inplace=True)
"""=====================================================================================================================
8 device_code1、device_code2和device3的处理, 安卓=1 iPhone=0
"""

"""【训练集】"""
phone1=DataFrame(train_operation['device_code3']).isnull()
phone_size=(phone1 == True).astype(bool).sum(axis=1)
phone2=DataFrame(list(zip(phone_size)))
iphone_android=phone2.rename(columns={0:"phone"})
train_operation['device_code3'] =iphone_android
train_operation.drop(['device_code1','device_code2'],axis=1,inplace=True)

"""【测试集】"""
phone3=DataFrame(test_operation['device_code3']).isnull()
phone_size=(phone3 == True).astype(bool).sum(axis=1)
phone4=DataFrame(list(zip(phone_size)))
iphone_android=phone4.rename(columns={0:"phone"})
test_operation['device_code3'] =iphone_android
test_operation.drop(['device_code1','device_code2'],axis=1,inplace=True)
"""=====================================================================================================================
9 MAC地址的处理
"""
"""【训练集】"""
mac1 = DataFrame(train_operation['mac1']).isnull()
mac_null=(mac1 == True).astype(bool).sum(axis=1)
mac2 = DataFrame(list(zip(mac_null)))
mac_is_null = mac2.rename(columns={0:"mac1_is_null"})
train_operation = train_operation.join(mac_is_null)
train_operation.drop(['mac1'],axis=1,inplace=True)

mac1 = DataFrame(train_operation['mac2']).isnull()
mac_null=(mac1 == True).astype(bool).sum(axis=1)
mac2 = DataFrame(list(zip(mac_null)))
mac_is_null = mac2.rename(columns={0:"mac2_is_null"})
train_operation = train_operation.join(mac_is_null)
train_operation.drop(['mac2'],axis=1,inplace=True)

"""【测试集】"""
mac1 = DataFrame(test_operation['mac1']).isnull()
mac_null=(mac1 == True).astype(bool).sum(axis=1)
mac2 = DataFrame(list(zip(mac_null)))
mac_is_null = mac2.rename(columns={0:"mac1_is_null"})
test_operation = test_operation.join(mac_is_null)
test_operation.drop(['mac1'],axis=1,inplace=True)

mac1 = DataFrame(test_operation['mac2']).isnull()
mac_null=(mac1 == True).astype(bool).sum(axis=1)
mac2 = DataFrame(list(zip(mac_null)))
mac_is_null = mac2.rename(columns={0:"mac2_is_null"})
test_operation = test_operation.join(mac_is_null)
test_operation.drop(['mac2'],axis=1,inplace=True)
"""=====================================================================================================================
10 ip地址的处理
"""
"""【训练集】"""
ip=DataFrame(train_operation['ip1']).isnull()
ip_null=(ip == True).astype(bool).sum(axis=1)
ip2=DataFrame(list(zip(ip_null)))
ip_is_null=ip2.rename(columns={0:"ip1_is_null"})
train_operation=train_operation.join(ip_is_null)
train_operation.drop(['ip1'],axis=1,inplace=True)

ip=DataFrame(train_operation['ip2']).isnull()
ip_null=(ip == True).astype(bool).sum(axis=1)
ip2=DataFrame(list(zip(ip_null)))
ip_is_null=ip2.rename(columns={0:"ip2_is_null"})
train_operation=train_operation.join(ip_is_null)
train_operation.drop(['ip2'],axis=1,inplace=True)
"""【测试集】"""
ip=DataFrame(test_operation['ip1']).isnull()
ip_null=(ip == True).astype(bool).sum(axis=1)
ip2=DataFrame(list(zip(ip_null)))
ip_is_null=ip2.rename(columns={0:"ip1_is_null"})
test_operation=test_operation.join(ip_is_null)
test_operation.drop(['ip1'],axis=1,inplace=True)

ip=DataFrame(test_operation['ip2']).isnull()
ip_null=(ip == True).astype(bool).sum(axis=1)
ip2=DataFrame(list(zip(ip_null)))
ip_is_null=ip2.rename(columns={0:"ip2_is_null"})
test_operation=test_operation.join(ip_is_null)
test_operation.drop(['ip2'],axis=1,inplace=True)
"""=====================================================================================================================
11 wifi的处理，判断是否为空
"""
"""【训练集】"""
wifi=DataFrame(train_operation['wifi']).isnull()
wifi_null=(wifi == True).astype(bool).sum(axis=1)
wifi2=DataFrame(list(zip(wifi_null)))
wifi_is_null=wifi2.rename(columns={0:"wifi_is_null"})
train_operation=train_operation.join(wifi_is_null)
train_operation.drop(['wifi'],axis=1,inplace=True)

"""【测试集】"""
wifi=DataFrame(test_operation['wifi']).isnull()
wifi_null=(wifi == True).astype(bool).sum(axis=1)
wifi2=DataFrame(list(zip(wifi_null)))
wifi_is_null=wifi2.rename(columns={0:"wifi_is_null"})
test_operation=test_operation.join(wifi_is_null)
test_operation.drop(['wifi'],axis=1,inplace=True)

"""=====================================================================================================================
12 地理位址"geocode"的处理
"""
"""【训练集】"""
train_operation["geo_code"]= train_operation["geo_code"].apply(lambda x: str(x))
train_operation["geo_code"]= train_operation["geo_code"].apply(lambda x: geohash.decode(x) if x != 'nan' else (200,200))
train_operation["geo_code"]= train_operation["geo_code"].apply(lambda x: list(x))

def address(x):
    w=200
    j=200
    if int(x[0])<200:
        #0-70/10=7
        if int(x[0])>70: w=8
        w=int(int(x[0])/10)
        #0-140/14=10
        j=int(int(x[1])/14)
    return str(w)+str(j)
train_operation["geo_code"]= train_operation["geo_code"].apply(lambda x: address(x))




"""【测试集】"""
test_operation["geo_code"]= test_operation["geo_code"].apply(lambda x: str(x))
test_operation["geo_code"]= test_operation["geo_code"].apply(lambda x: geohash.decode(x) if x != 'nan' else (200,200))
test_operation["geo_code"]= test_operation["geo_code"].apply(lambda x: list(x))

test_operation["geo_code"]= test_operation["geo_code"].apply(lambda x: address(x))

"""合并"""

n1=set(train_operation["geo_code"])
n2=set(test_operation["geo_code"])
n=list(n1.union(n2))
dic={}
for i,j in enumerate(n):
    dic[j]=i

train_operation["geo_code"]= train_operation["geo_code"].apply(lambda x: str(x))
train_operation["geo_code"]= train_operation["geo_code"].map(dic)

test_operation["geo_code"]= test_operation["geo_code"].apply(lambda x: str(x))
test_operation["geo_code"]= test_operation["geo_code"].map(dic)

"""=====================================================================================================================
13 ip_sub的处理,判断ip1_sub是否为空，删除ip2_sub
"""
ip=DataFrame(train_operation['ip1_sub']).isnull()
ip_null=(ip == True).astype(bool).sum(axis=1)
ip2=DataFrame(list(zip(ip_null)))
ip_is_null=ip2.rename(columns={0:"ip1_sub_is_null"})
train_operation=train_operation.join(ip_is_null)
train_operation.drop(['ip1_sub'],axis=1,inplace=True)

ip=DataFrame(test_operation['ip1_sub']).isnull()
ip_null=(ip == True).astype(bool).sum(axis=1)
ip2=DataFrame(list(zip(ip_null)))
ip_is_null=ip2.rename(columns={0:"ip1_sub_is_null"})
test_operation=test_operation.join(ip_is_null)
test_operation.drop(['ip1_sub'],axis=1,inplace=True)

train_operation.drop(['ip2_sub'],axis=1,inplace=True)
test_operation.drop(['ip2_sub'],axis=1,inplace=True)
"""
14 保存数据
"""

"""【训练集】"""
train_operation.to_csv(path + 'train_operation_V1.csv',index=False)

"""【测试集】"""
test_operation.to_csv(path + 'test_operation_V1.csv',index=False)